In [6]:
import numpy as np
import matplotlib.pyplot as plt

from qibolab import create_platform, ExecutionParameters, AveragingMode, AcquisitionType
from qibolab.pulses import PulseSequence
from simple_normalize import normalize_to_minus_one_one

In [7]:
import numpy as np
from qdp import DensityMatrixExponentiation
platform = create_platform("dummy")
my_protocol = DensityMatrixExponentiation(theta=np.pi,N=3,num_work_qubits=1,num_instruction_qubits=3,number_muq_per_call=1)
my_protocol.memory_call_circuit(num_instruction_qubits_per_query=3)
print('DME, q0 is target qubit, q1,q2 and q3 are instruction qubit')
print(my_protocol.c.draw())
c = my_protocol.c

[Qibo 0.2.3|ERROR|2024-04-25 15:04:55]: Platform icarusq_iqm5q does not exist.


ValueError: Platform icarusq_iqm5q does not exist.